In [1]:
import requests
import traceback
import json
import csv
import re 

In [9]:
# Build a simple csv file
# Data to be written to the CSV file
rows = [
    ["employee_name", "age", "city"],
    ["alice", "10.1", "new york"],
    ["bob", "11", "los angeles"],
    ["charlie", "12", "chicago"],
    ["john", "14.5", "soperton"],
    ["mike", "17.25", "dallas"]
]
rows

[['employee_name', 'age', 'city'],
 ['alice', '10.1', 'new york'],
 ['bob', '11', 'los angeles'],
 ['charlie', '12', 'chicago'],
 ['john', '14.5', 'soperton'],
 ['mike', '17.25', 'dallas']]

In [10]:
# Write the data file
file_path = '/home/ubuntu/ragllm2/data/employee_data.csv'

# Open a file in write mode
with open(file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    
    # Writing rows
    for row in rows:
        writer.writerow(row)

In [6]:
# Now let reprocess the data files
import requests

# Define the URL to which the form submits data
# Assuming 'base_url' is the domain where the form resides, you need to replace it with the actual domain
base_url = 'http://localhost:5000'
endpoint = '/reprocess'
url = base_url + endpoint

# Make the POST request to the endpoint
response = requests.post(url)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful... Context updated")
else:
    print(f"Request failed. Status code: {response.status_code}")


Request successful... Context updated


In [7]:
# Now lets query the model
# The URL to which the request is being sent
url = 'http://localhost:5000/query'

# Headers to be sent with the request
headers = {
    'Content-Type': 'application/json'
}

# Data to be sent in the request
data = {
    'query': 'What is Johns age?'
}

# Make the POST request
response = requests.post(url, headers=headers, data=json.dumps(data))

# Check the response
if response.status_code == 200:
    print("Success:", response.text)
else:
    print("Error:", response.status_code, response.text)

Success: {"qhid":"07468275-eefb-4ec4-84e8-c2b95dce5ec6","response":"14.5"}



In [58]:
# Get the query history id - qhid
qhid = response.json()['qhid']
qhid

'f3eada53-d769-430f-9ba9-a5573e7a0dea'

In [59]:
# Now lets cointinue the discussion using the quid
# The URL to which the request is being sent
url = 'http://localhost:5000/query'

# Headers to be sent with the request
headers = {
    'Content-Type': 'application/json'
}

# Data to be sent in the request
data = {
    'query': 'What city does he live in?',
    'qhid': qhid
}

# Make the POST request
response = requests.post(url, headers=headers, data=json.dumps(data))

# Check the response
if response.status_code == 200:
    print("Success:", response.text)
else:
    print("Error:", response.status_code, response.text)

Success: {"qhid":"f3eada53-d769-430f-9ba9-a5573e7a0dea","response":""}



In [52]:
# Now lets plot some data
# The URL to which the request is being sent
url = 'http://localhost:5000/query'

# Headers to be sent with the request
headers = {
    'Content-Type': 'application/json'
}

# Data to be sent in the request
data = {
    'query': 'Generate python code to plot employees by age from the context.',
    'qhid': qhid
}

# Make the POST request
response = requests.post(url, headers=headers, data=json.dumps(data))

# Check the response
if response.status_code == 200:
    print("Success:", response.text)
else:
    print("Error:", response.status_code, response.text)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [51]:
# Convert to json data
response_data = json.loads(response.text)

# Extracting the code string
code_string = response_data['response'].split("```python\n", 1)[1].rsplit("\n```", 1)[0]

# Function to correct the code
def correct_code(code):
    corrected_lines = []
    for line in code.split('\n'):
        # Detect lines that might be comments but are not marked as such
        if re.match(r"^[A-Z].*", line) and not re.match(r"^(import |from |class |def |#)", line):
            line = "# " + line
        corrected_lines.append(line)
    
    return '\n'.join(corrected_lines)

# Apply the correction
corrected_code = correct_code(code_string)
print(corrected_code)

import pandas as pd
import matplotlib.pyplot as plt

# Create a DataFrame from the given data

data = {'employee_name': ['alice', 'john', 'charlie', 'bob'],
        'age': [10.1, 14.5, 12, 11],
        'city': ['new york', 'soperton', 'chicago', 'los angeles']}
df = pd.DataFrame(data)

 Plot the employees by age using a bar chart

plt.bar(df['employee_name'], df['age'])
plt.xlabel('Employee Name')
plt.ylabel('Age')
plt.title('Employees plotted by Age')
plt.show()


In [15]:
exec(corrected_code)

IndentationError: unexpected indent (<string>, line 15)